In [1]:
# recommended to use virtual env to install pip package
# see link official docs for installation reference: https://github.com/googleapis/python-analytics-data#installation
%pip install google-analytics-data
%pip install sqlalchemy
%pip install pyobdc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pyobdc (from versions: none)
ERROR: No matching distribution found for pyobdc
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

import os  
os.makedirs('tests/', exist_ok=True)

In [3]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    OrderBy,
)

In [4]:
# modified from official google docs for GA4: 
# https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries

def pull_from_ga_into_df(dimensions, metrics, order_by):
    """Runs a simple report on a Google Analytics 4 property."""
    # TODO(developer): Uncomment this variable and replace with your
    #  Google Analytics 4 property ID before running the sample.
    # property_id = "YOUR-GA4-PROPERTY-ID"

    # Using a default constructor instructs the client to use the credentials
    # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
    client = BetaAnalyticsDataClient()
    property_id="353368209"

    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name=d) for d in dimensions],
        metrics=[Metric(name=m) for m in metrics],
        date_ranges=[DateRange(start_date="30daysAgo", end_date="today")],
        order_bys=[OrderBy(desc=True, dimension=OrderBy.DimensionOrderBy(dimension_name=order_by))]
    )
    response = client.run_report(request)
    
    ## process into pd dataframe
    
    # get headers
    data_dict = {}
    type_dict = {}
    for d in response.dimension_headers:
        data_dict[d.name] = []

    for m in response.metric_headers:
        data_dict[m.name] = []
        type_dict[m.name] = m.type_

    # insert into dictionary
    for r in response.rows:
        vals = [v.value for v in [*r.dimension_values, *r.metric_values]]
        for i, key in enumerate(data_dict):
            to_add = vals[i]
            if key in type_dict:
                to_add = int(vals[i]) if type_dict[key] == 1 else float(vals[i])
            data_dict[key].append(to_add)

    df = pd.DataFrame(data=data_dict)
    
    return df

In [5]:
def cast_types(df, type_map):
    pass

In [6]:
query_map = {}

In [7]:
query_map["AudienceOverview"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "newUsers",
        "sessionsPerUser",
        "screenPageViewsPerSession",
        "screenPageViews",
        "sessions",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "hostName"
    ],
    "order_by": "date"
}

In [8]:
audience_overview_df = pull_from_ga_into_df(
    query_map["AudienceOverview"]["dimensions"], 
    query_map["AudienceOverview"]["metrics"],
    query_map["AudienceOverview"]["order_by"],
)
audience_overview_df.to_csv("tests/AudienceOverview.csv", index=False)
audience_overview_df

,date,hostName,averageSessionDuration,bounceRate,newUsers,sessionsPerUser,screenPageViewsPerSession,screenPageViews,sessions,activeUsers
0,20230310,moandbear.com.ph,3.524283,1.000000,3,1.500000,1.166667,7,6,4
1,20230309,moandbear.com.ph,46.534410,0.692308,10,1.083333,1.461538,19,13,12
2,20230308,moandbear.com.ph,90.478372,0.307692,10,1.083333,2.384615,31,13,12
3,20230307,moandbear.com.ph,356.839097,0.333333,6,1.000000,2.333333,14,6,6
4,20230306,connectsys.shop,507.054196,0.500000,6,1.000000,2.166667,13,6,6
5,20230306,moandbear.com.ph,13.906948,0.666667,9,1.500000,1.000000,15,15,10
6,20230305,connectsys.shop,24.650139,0.500000,1,2.000000,1.000000,2,2,1
7,20230305,moandbear.com.ph,11.933743,0.600000,3,1.666667,1.200000,6,5,3
8,20230304,moandbear.com.ph,421.267917,0.333333,2,1.500000,2.333333,7,3,2
9,20230304,connectsys.shop,17.537123,0.333333,0,3.000000,1.666667,5,3,1


In [9]:
query_map["AudienceByHour"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "newUsers",
        "sessionsPerUser",
        "screenPageViewsPerSession",
        "screenPageViews",
        "sessions",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "dateHour",
        "hostName",
    ],
    "order_by": "dateHour",
}

In [10]:
audience_by_hour_df = pull_from_ga_into_df(
    query_map["AudienceByHour"]["dimensions"], 
    query_map["AudienceByHour"]["metrics"],
    query_map["AudienceByHour"]["order_by"],
)
audience_by_hour_df.to_csv("tests/AudienceByHour.csv", index=False)
audience_by_hour_df

,date,dateHour,hostName,averageSessionDuration,bounceRate,newUsers,sessionsPerUser,screenPageViewsPerSession,screenPageViews,sessions,activeUsers
0,20230310,2023031012,moandbear.com.ph,0.000000,1.000000,1,1.0,1.0,2,2,2
1,20230310,2023031011,moandbear.com.ph,0.000000,1.000000,0,1.0,1.0,1,1,1
2,20230310,2023031006,moandbear.com.ph,0.000000,1.000000,1,0.0,1.0,1,1,0
3,20230310,2023031005,moandbear.com.ph,21.145696,1.000000,1,1.0,2.0,2,1,1
4,20230310,2023031001,moandbear.com.ph,0.000000,1.000000,0,1.0,1.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
165,20230222,2023022200,uat.moandbear.com.ph,172.744891,0.000000,1,1.0,2.0,2,1,1
166,20230221,2023022122,uat.moandbear.com.ph,1210.965848,1.000000,0,1.0,3.0,3,1,1
167,20230221,2023022122,moandbear.com.ph,13.927580,0.500000,1,1.0,1.5,3,2,2
168,20230221,2023022122,uat.connectsys.shop,5.643172,1.000000,1,1.0,1.0,1,1,1


In [11]:
query_map["AudienceByCategory"] = {
    "metrics": [
        "newUsers",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "country",
        "deviceCategory",
        "language",
#         "pagePath", ## this makes the request incompatible
    ],
    "order_by": "date",
}

In [12]:
audience_by_category_df = pull_from_ga_into_df(
    query_map["AudienceByCategory"]["dimensions"], 
    query_map["AudienceByCategory"]["metrics"],
    query_map["AudienceByCategory"]["order_by"],
)
audience_by_category_df.to_csv("tests/AudienceByCategory.csv", index=False)
audience_by_category_df

,date,country,deviceCategory,language,newUsers,activeUsers
0,20230310,Philippines,mobile,English,2,3
1,20230310,United States,desktop,English,1,0
2,20230310,Philippines,tablet,English,0,1
3,20230309,Philippines,mobile,English,5,6
4,20230309,Philippines,desktop,English,2,2
...,...,...,...,...,...,...
85,20230221,Philippines,desktop,English,4,4
86,20230221,Philippines,mobile,English,2,2
87,20230221,Singapore,desktop,English,1,1
88,20230221,Singapore,mobile,English,1,2


In [13]:
query_map["GaOrders"] = {
    "metrics": [
        "purchaseRevenue",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "month",
        "date",
        "transactionId",
        "campaignName",
    ],
    "order_by": "campaignName",
}

In [14]:
ga_orders_df = pull_from_ga_into_df(
    query_map["GaOrders"]["dimensions"], 
    query_map["GaOrders"]["metrics"],
    query_map["GaOrders"]["order_by"],
)
ga_orders_df.to_csv("tests/GaOrders.csv", index=False)
ga_orders_df

,month,date,transactionId,campaignName,purchaseRevenue,activeUsers
0,03,20230308,384,Mo and Bear Instagram,501.556972,1
1,02,20230228,382,(referral),0.000000,1
2,02,20230228,427,(referral),0.000000,1
3,02,20230224,380,(direct),0.000000,1
4,02,20230225,422,(direct),0.000000,1
5,02,20230228,424,(direct),0.000000,1
6,02,20230228,426,(direct),0.000000,1
7,02,20230228,428,(direct),0.000000,1
8,02,20230228,429,(direct),0.000000,1
9,02,20230228,430,(direct),0.000000,1


In [15]:
query_map["PageTitleViews"] = {
    "metrics": [
        "screenPageViews", # not sure either
    ],
    "dimensions": [
        "date",
        "hostName",
        "landingPagePlusQueryString",
        "pagePath", # could alternatively be pageLocation
        "pageTitle",
        "percentScrolled", # assuming this is page depth
    ],
    "order_by": "date",
}

In [16]:
page_title_views_df = pull_from_ga_into_df(
    query_map["PageTitleViews"]["dimensions"], 
    query_map["PageTitleViews"]["metrics"],
    query_map["PageTitleViews"]["order_by"],
)
page_title_views_df.to_csv("tests/PageTitleViews.csv", index=False)
page_title_views_df

,date,hostName,landingPagePlusQueryString,pagePath,pageTitle,percentScrolled,screenPageViews
0,20230310,moandbear.com.ph,/,/,Shopping Cart Software & Ecommerce Software So...,,3
1,20230310,moandbear.com.ph,/orders/,/orders/,Orders,,2
2,20230310,moandbear.com.ph,/?fbclid=PAAabCu8MfAQKGlAOY-1GGeC-F_izq94OVBYU...,/,Shopping Cart Software & Ecommerce Software So...,,1
3,20230310,moandbear.com.ph,/pet-care/,/pet-care/,Pet Care,,1
4,20230309,moandbear.com.ph,/,/,Shopping Cart Software & Ecommerce Software So...,,8
...,...,...,...,...,...,...,...
330,20230221,moandbear.com.ph,/pet-care/pet-food/?fbclid=IwAR3qyITVwt2bp3p6k...,/pet-care/pet-food/,Pet Care :: Pet Food - Mo&Bear Categories,,1
331,20230221,uat.connectsys.shop,/vendor,/vendor,Page Not Found,,1
332,20230221,uat.moandbear.com.ph,/brands/happy-life/,/,Shopping Cart Software & Ecommerce Software So...,,1
333,20230221,uat.moandbear.com.ph,/brands/happy-life/,/brands/happy-life/,Happy Life,,1


In [17]:
query_map["Acquisition"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "totalRevenue",
        "screenPageViewsPerSession",
        "sessions",
        "newUsers",
        "totalUsers",
    ],
    "dimensions": [
        "date",
#         "campaignName",
        "month",
        "transactionId",
#         "sourceMedium",
        "adSourceName", # alternatively firstUserGoogleAdsAdNetworkType, 
        # googleAdsAdNetworkType, sessionGoogleAdsAdNetworkType
        "hostName",
    ],
    "order_by": "date",
}

In [18]:
acquisition_df = pull_from_ga_into_df(
    query_map["Acquisition"]["dimensions"], 
    query_map["Acquisition"]["metrics"],
    query_map["Acquisition"]["order_by"],
)
acquisition_df.to_csv("tests/Acquisition.csv", index=False)
acquisition_df

,date,month,transactionId,adSourceName,hostName,averageSessionDuration,bounceRate,totalRevenue,screenPageViewsPerSession,sessions,newUsers,totalUsers
0,20230310,03,,,moandbear.com.ph,3.524283,1.000000,0.000000,1.166667,6,3,5
1,20230309,03,(not set),,moandbear.com.ph,46.534410,0.692308,0.000000,1.461538,13,10,12
2,20230308,03,(not set),,moandbear.com.ph,90.476962,0.307692,0.000000,2.384615,13,10,13
3,20230308,03,384,,moandbear.com.ph,0.018327,1.000000,501.557137,0.000000,1,0,1
4,20230307,03,(not set),,moandbear.com.ph,356.839097,0.333333,0.000000,2.333333,6,6,6
5,20230306,03,(not set),,connectsys.shop,507.054196,0.500000,0.000000,2.166667,6,6,6
6,20230306,03,(not set),,moandbear.com.ph,13.906948,0.666667,0.000000,1.000000,15,9,12
7,20230305,03,(not set),,connectsys.shop,24.650139,0.500000,0.000000,1.000000,2,1,2
8,20230305,03,(not set),,moandbear.com.ph,11.933743,0.600000,0.000000,1.200000,5,3,4
9,20230304,03,(not set),,moandbear.com.ph,421.267917,0.333333,0.000000,2.333333,3,2,2


In [19]:
# in memory test db
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [20]:
# # actual db

# from sqlalchemy import create_engine
# from sqlalchemy.engine import URL
# connection_url = URL.create(
#     "mssql+pyodbc",
#     username="ucdaadmin",
#     password="f@xgYW4A){Fm4>{@",
#     host="ucda.database.windows.net",
#     port=1433,
#     database="ucda",
#     query={
#         "driver": "ODBC Driver 18 for SQL Server",
#         "TrustServerCertificate": "yes",
#         "authentication": "ActiveDirectoryIntegrated",
#     },
# )

# engine = create_engine(connection_url)

In [21]:
audience_overview_df.to_sql('ga_audience_overview', con=engine, index=False)
audience_by_hour_df.to_sql('ga_audience_by_hour', con=engine, index=False)
audience_by_category_df.to_sql('ga_audience_by_category', con=engine, index=False)
page_title_views_df.to_sql('ga_page_title_views', con=engine, index=False)
ga_orders_df.to_sql('ga_orders', con=engine, index=False)
acquisition_df.to_sql('ga_acquisition', con=engine, index=False)

48

In [22]:
engine.execute("SELECT * FROM users").fetchall()

AttributeError: 'Engine' object has no attribute 'execute'